In [24]:
#update an address (top 25)

from pymongo import MongoClient

dbconn = MongoClient()
db = dbconn.parking.latlong

#db.remove_one({"collaped_address":collapsed_address})

collapsed_address = "200 DUPONT ST"

#db.ensureIndex({'collaped_address' : collapsed_address}, {unique : true, dropDups : true})

#collapsed_address = "600 BLOOR ST W"
req = {"collaped_address": collapsed_address}
resp = db.find(req)
for res in resp:
    print(res)

#db.remove({"collaped_address":collapsed_address, "num_associated":98})

{'latitude': 43.675614, 'addresses_associated': ['196 DUPONT ST', '132 DUPONT ST', '195 DUPONT ST', '102 DUPONT ST', '165 DUPONT ST', '124 DUPONT ST', '186 DUPONT ST', '108 DUPONT ST', '112 DUPONT ST', '176 DUPONT ST', '167 DUPONT ST', '168 DUPONT ST', '150 DUPONT ST', '116 DUPONT ST', '151 DUPONT ST', '103 DUPONT ST', '189 DUPONT ST', '161 DUPONT ST', '178 DUPONT ST', '142 DUPONT ST', '160 DUPONT ST', '125 DUPONT ST', '162 DUPONT ST', '126 DUPONT ST', '134 DUPONT ST', '119 DUPONT ST', '179 DUPONT ST', '198 DUPONT ST', '106 DUPONT ST', '164 DUPONT ST', '104 DUPONT ST', '107 DUPONT ST', '200 DUPONT ST', '117 DUPONT ST', '118 DUPONT ST', '121 DUPONT ST', '133 DUPONT ST', '101 DUPONT ST', '109 DUPONT ST', '114 DUPONT ST', '191 DUPONT ST', '188 DUPONT ST', '158 DUPONT ST', '184 DUPONT ST', '123 DUPONT ST', '144 DUPONT ST', '185 DUPONT ST', '190 DUPONT ST', '194 DUPONT ST', '120 DUPONT ST', '199 DUPONT ST', '143 DUPONT ST', '180 DUPONT ST', '131 DUPONT ST', '163 DUPONT ST', '128 DUPONT ST',

In [ ]:
def get_location_stats('collapsed_address'):
    #find mapped addresses in db
    req = {"collaped_address": collapsed_address}
    resp = db.find(req)
    addresses_to_find = resp['addresses_associated']
    

In [77]:
key = {'collaped_address': collapsed_address}
changes = {'$set':{'latitude':43.6478110, 'longitude':-79.3911180}}
    
print(db.update_one(key,changes))

In [ ]:
#ROUND TO A STREET ADDRESS
import csv
import sys
from collections import defaultdict
import re

#testing address resolution
def myround(x, base=100):
    return int(base * round(float(x)/base))

def get_number(address):
    numbers_in_address = [int(s) for s in address.split() if s.isdigit()]
    if numbers_in_address:
        return numbers_in_address[0]
    else:
        return None

def collapse_to_street(address):
    return re.sub("^\d+\s|\s\d+\s|\s\d+$", "", address)

csv.field_size_limit(sys.maxsize)
years=['2008','2009','2010','2011','2012','2013','2014','2015']
files_to_open=['data/'+year+'.csv' for year in years]
addresses100 = defaultdict(set)
for idx,file in enumerate(files_to_open):
    with open(file, 'rt', encoding='utf8') as f:
        reader = csv.DictReader(f)
        for line in reader:
            address = get_number(line['location2'])
            if address:
                addresses100[collapse_to_street(line['location2'])].add(myround(address))
    print("PROCESSED "+file)

In [10]:
#get all collapsed address/lat-long pairs
from pymongo import MongoClient

dbconn = MongoClient()
db = dbconn.parking.latlong

all_records = db.find({},{"collaped_address":1,"latitude":1,"longitude":1, "num_associated":1, '_id':0})

ar = []

for record in all_records:
    ar.append(record)

In [11]:
print(ar[0])

{'latitude': 43.61593765, 'collaped_address': '800 BAY', 'longitude': -79.321369, 'num_associated': 3}


In [ ]:
import fiona
from shapely import geometry

db = dbconn.parking.censustract

with fiona.open("gct_000b11a_e/gct_000b11a_e.shp") as fiona_collection:
    
    ct_map = {}
    ct_map_num = {}

    # In this case, we'll assume the shapefile only has one record/layer (e.g., the shapefile
    # is just for the borders of a single country, etc.).
    while fiona_collection:
        shapefile_record = fiona_collection.next()
        if shapefile_record['properties']['CMANAME']=='Toronto':
            census_tract = shapefile_record['properties']['CTUID']
            shape = geometry.asShape( shapefile_record['geometry'] )
            not_found = []
            ct_map[census_tract]=[]
            ct_map_num[census_tract]=0
            print(census_tract)
            for address in ar:
                point = geometry.Point(address['longitude'], address['latitude'])
                if shape.contains(point):
                    ct_map_num[census_tract]+=address['num_associated']
                    ct_map[census_tract].append(address['collaped_address'])
                else:
                    not_found.append(address)
            ar = not_found

5350091.02
5350128.02
5350412.15
5350341.04
5350363.04
5350513.01
5350403.07
5350442.03
5350377.07
5350412.23
5350140.00
5350591.01
5350576.17
5350311.05
5350244.02
5350564.02
5350003.00
5350018.00
5350442.04
5350576.09
5350613.04
5350411.24
5350527.06
5350030.00
5350317.02
5350279.02
5350576.06
5350029.00
5350312.03
5350442.06
5350304.02
5350800.02
5350362.02
5350576.35
5350576.33
5350820.01
5350500.02
5350412.14
5350366.00
5350363.05
5350171.00
5350502.02
5350068.00
5350357.01
5350119.00
5350106.00
5350410.13
5350024.00
5350365.00
5350810.01
5350353.04
5350377.04
5350410.02
5350509.02
5350123.00
5350612.10
5350136.01
5350622.00
5350401.17
5350376.08
5350169.02
5350250.05
5350193.00
5350402.08
5350411.15
5350635.00
5350587.02
5350324.06
5350098.00
5350102.02
5350520.08
5350261.00
5350528.41
5350805.02
5350135.00
5350442.02
5350576.20
5350527.09
5350639.00
5350401.08
5350229.00
5350362.01
5350341.03
5350476.00
5350176.00
5350528.39
5350403.05
5350564.01
5350377.02
5350224.00
5350210.00